In [1]:
from itertools import product

import numpy as np
from aocd import get_data, submit

DAY = 8
YEAR = 2022

In [48]:
# use real data
raw = get_data(day=DAY, year=YEAR)

# print(raw)


In [49]:
def parse_data(data):
    data = data.split("\n")
    data = np.array([list(d) for d in data], dtype=int)
    return data


data = parse_data(raw)


# Part 1

### long and stupid

In [4]:
mask = np.zeros_like(data)
mask[0], mask[-1], mask[:, 0], mask[:, -1] = True, True, True, True


def tree_mask(data, mask, flip=False):
    if flip:
        mask = np.transpose(mask)
        data = np.transpose(data)

    n, m = data.shape
    for idx in range(1, n - 1):
        line = data[idx]
        mh = line[0]
        for jdx, el in enumerate(line[1:]):
            if el > mh:
                mask[idx][jdx + 1] |= True
                mh = el
        line = line[::-1]
        mh = line[0]
        for jdx, el in enumerate(line[1:]):
            if el > mh:
                mask[idx][m - jdx - 2] |= True
                mh = el

    if flip:
        mask = np.transpose(mask)

    return mask


mask = tree_mask(data, mask, False)
mask = tree_mask(data, mask, True)

result = np.count_nonzero(mask)
result


1835

## shorter, but non-intuitive

In [5]:
mask = np.zeros_like(data)
mask[0], mask[-1], mask[:, 0], mask[:, -1] = True, True, True, True

# find strictly increasing values in each direction
mask |= np.diff(np.maximum.accumulate(data, axis=-1), axis=-1, prepend=1) > 0
mask |= (np.diff(np.maximum.accumulate(data[:, ::-1], axis=-1), axis=-1, prepend=1) > 0)[:, ::-1]
mask |= np.diff(np.maximum.accumulate(data, axis=0), axis=0, prepend=1) > 0
mask |= (np.diff(np.maximum.accumulate(data[::-1], axis=0), axis=0, prepend=1) > 0)[::-1]

result = np.count_nonzero(mask)
result


1835

### simplest

In [24]:
mask = np.zeros_like(data)
n, m = data.shape

for idx in range(n):
    row = data[idx]
    for jdx in range(m):
        col = data[:, jdx]

        if jdx == 0 or jdx == m - 1 or idx == 0 or idx == n - 1:
            mask[idx, jdx] = True
            continue

        el = data[idx, jdx]
        if (
            el > np.max(row[:jdx])
            or el > np.max(row[jdx + 1 :])
            or el > np.max(col[:idx])
            or el > np.max(col[idx + 1 :])
        ):
            mask[idx, jdx] |= True

result = np.count_nonzero(mask)
result


1835

In [18]:
# submit(result, part="a", day=DAY, year=YEAR)

# Part 2

### non-elegant

In [67]:
def extract_dist(data, scores, flip=False):
    if flip:
        scores = np.transpose(scores)
        data = np.transpose(data)

    n, m = data.shape
    for idx in range(1, n - 1):
        for jdx in range(1, m - 1):
            row = data[idx]
            mh = data[idx][jdx]
            if scores[idx][jdx] is None:
                scores[idx][jdx] = []

            score = 0
            for el in row[jdx + 1 :]:
                score += 1
                if mh <= el:
                    break
            scores[idx][jdx].append(score)

            score = 0
            for el in row[:jdx][::-1]:
                score += 1
                if mh <= el:
                    break
            scores[idx][jdx].append(score)

    if flip:
        scores = np.transpose(scores)

    return scores


def calc_score(data):
    scores = np.zeros_like(data, dtype=int)
    dist = scores = np.full(data.shape, None)
    dist = extract_dist(data, dist)
    dist = extract_dist(data, dist, True)

    n, m = data.shape
    for idx in range(n):
        for jdx in range(m):
            if dist[idx][jdx] is None:
                scores[idx][jdx] = 0
            else:
                scores[idx, jdx] = np.prod(dist[idx, jdx])
    return dist.astype(int)


result = np.max(calc_score(data))


### shorter, bit complex

In [25]:
def calc_score(x, i, j):
    c = x[i, j]
    n, m = x.shape

    # directional slices
    slice_list = [
        [slice(i - 1, None, -1), j],  # up
        [i, slice(j - 1, None, -1)],  # left
        [i, slice(j + 1, None)],  # right
        [slice(i + 1, None), j],  #  down
    ]

    # get data for all directions
    dist_list = [x[slc[0], slc[1]] for slc in slice_list]

    # cut short when first equal tree
    dist_list = [l[: np.argwhere(l >= c)[0, 0] + 1 if np.any(l >= c) else None] for l in dist_list]

    return np.prod([len(l) for l in dist_list])


scores = []
n, m = data.shape
for idx in range(1, n - 1):
    for jdx in range(1, m - 1):
        scores.append(calc_score(data, idx, jdx))

result = max(scores)
result


263670

### simplest

In [51]:
scores = np.ones_like(data)
n, m = data.shape

for idx in range(n):
    row = data[idx]
    for jdx in range(m):
        col = data[:, jdx]
        el = data[idx, jdx]

        score = []
        trees_array = [col[:idx][::-1], col[idx + 1 :], row[:jdx][::-1], row[jdx + 1 :]]  # up, down, left, right

        for trees in trees_array:
            num = 0
            for t in trees:
                num += 1
                if t >= el:
                    break
            score.append(num)

        scores[idx, jdx] = np.prod(score)

result = np.max(scores)
result


263670

In [52]:
# submit(result, part="b", day=DAY, year=YEAR)